Блокнот является развитием блокнота Bayesian Networks with pyAgrum (сначала лучше ознакомиться с ним) и рассматривает усложненную задачу из области медицины, в которой помимо наблюдаемых симптомов можно совершать еще и диагностические действия, направленные на получение дополнительной информации о состоянии пациента, а также выбирать вид терапии.

Условие модицифированной задачи следующее:

Пациент может быть здоров, либо у него может быть наблюдаться одно из взаимоисключающих заболеваний - З1 или З2, априорные вероятности которых $0.01$ и $0.006$ соответственно. Оба заболевания могут вызывать симтомы С1 и С2. Вероятности появления симптома С1 приведены в таблице:

| Болезнь |  Вероятность  |
|---------|---------------|
|  Нет    |  0.05         |
|  З1     |  0.9          |
|  З2     |  0.4          |

Вероятности появления симптома С2 приведены в таблице:

| Болезнь |  Вероятность  |
|---------|---------------|
|  Нет    |  0.05         |
|  З1     |  0.3          |
|  З2     |  0.8          |

Существует также диагностическая процедура (тест), которая позволяет выявить заболевание З2 (как это обычно бывает, с некоторой точностью) - если у пациента действительно присутствует заболевание З2, то результат процедуры оказывается положительным в $95\%$ случаев, однако он также оказывается положительным в $3\%$ случаев, когда у пациента нет З2. Стоимость теста - 1200 руб.

Кроме того, для данной группы заболеваний существует два (взаимоисключающих) вида терапии - Т1 и Т2 (стоимостью 2500 руб. и 3000 руб. соответственно). Первая оказывается эффективна для лечения З1, вторая - для З2. Известны также значения стоимости каждого из видов терапии и эффективности, отражающей субъективные неудобства от каждого возможного развития событий:

| Болезнь | Терапия  |  Эффект  |
|---------|----------|----------|
|  Нет    |  Нет     |     0    |
|  Нет    |  Т1      |   -20    |   
|  Нет    |  Т2      |   -25    |
|  З1     |  Нет     |   -100   |
|  З1     |  T1      |   -50    |
|  З1     |  T2      |   -125   |
|  З2     |  Нет     |   -110   |
|  З2     |  T1      |   -130   |
|  З2     |  T2      |   -45    |

Для описания данной задачи воспользуемся возможностями библиотеки pyAgrum. Построим модель (байесовскую сеть), а потом ответим с ее помощью на несколько вопросов, касающихся выбора оптимальной стратегии для различных ситуаций.

Примечание! Вообще-то pyAgrum поддерживает возможность создания диаграмм влияния, но в текущих версиях вывод на них работает не совсем корректно. Учитывая, что диаграммы влияния являются расширением байесовских сетей, в данном блокноте мы будем использовать байесовские сети для вычисления вероятностей сложных событий, а часть, связанную с определением (ожидаемых) полезностей и оптимальных управлений будем осуществлять вручную.


In [1]:
import os

%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt

In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

# Создание байесовской сети

Если строить данную задачу как диаграмму влияния, но следовало бы внести две вершины принятия решения (проводить тест или нет) и несколько (возможны варианты) вершин полезности. Как указывалось выше, в этом блокноте мы моделируем только вероятностный вывод, оставляя всю обработку, связанную с принятием решений и за рамками самой сети (весь соответствующий анализ будет проводиться программным кодом, внешним по отношению к сети).

Таким образом, вершинами байесовской сети должны стать переменные, описывающие диагностическую ситуацию. Общая схема сети будет включать вершину, связанную с наличием у пациента болезни (значение этой вершины не будет наблюдаемым, мы будем лишь строить распределение вероятностей значения соответствущей переменной в зависимости от имеющейся информации), а также вершины, связанные с доступной врачу диагностической информацией: симптом С1, симптом С2 и результат теста (Д). Следует отметить, что формально результат теста не всегда доступен, однако факт его доступности или недоступности будет обрабатываться внешним кодом.


In [3]:
medicine_bn = gum.BayesNet()
print(medicine_bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


## Переменные (и вероятностные узлы)

pyAgrum поддерживает несколько видов переменных. Для данной задачи проще всего использовать, так называемые, `LabelizedVariable`, то есть переменные, которые могут принимать одно из фиксированного набора значений, каждое из которых имеет свою метку. Например, переменная Disease (Болезнь) может принимать одно из трех значений 'Нет', 'З1' и 'З2'.


In [4]:
# Создадим переменную 
# Первый параметр - имя переменной, второй - опциональный комментарий, а третий - количство значений
va = gum.LabelizedVariable('Disease', 'Is the patient sick?', 3)
va.changeLabel(0, 'No')
va.changeLabel(1, 'Disease1')
va.changeLabel(2, 'Disease2')
va

(gum::LabelizedVariable@0x203c9f5b0d0) Disease<No,Disease1,Disease2>

In [5]:
medicine_bn.add(va)

0

Аналогично создадим и другие переменные:

In [6]:
# Симптом 1
medicine_bn.add(gum.LabelizedVariable('Symptom1', 'Does the patient have the symptom 1?', 2))
# Симптом 2
medicine_bn.add(gum.LabelizedVariable('Symptom2', 'Does the patient have the symptom 2?', 2))
# Тест
medicine_bn.add(gum.LabelizedVariable('Test', 'Diagnostic test result', 2))


3

## Дуги

Необходимо связать узлы, соответствующие созданным переменным в струкуру, отражающую их взаимосвязь:

In [7]:
# Наличие болезни влияет на проявление симптомов и на результат диагностики:
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom1'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom2'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Test'))

In [8]:
medicine_bn

(gum::BayesNet<double>@0x203c9f5a800) BN{nodes: 4, arcs: 3, domainSize: 24, dim: 21}

## Таблицы условных вероятностей


In [9]:
medicine_bn.cpt(medicine_bn.idFromName('Disease')).fillWith([1-0.01-0.006, 0.01, 0.006])
medicine_bn.cpt(medicine_bn.idFromName('Disease'))

(gum::Potential<double>@0x203c9f5b840) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.9840  | 0.0100  | 0.0060  |

Несколько более сложный случай - задание условных вероятностей для вершин, у которых есть входящие дуги (таблицы таких вершин, очевидно, включают все возможные значения переменных, от которых они зависят). Для правильной установки нужно понимать порядок, в котором упомянуты переменные в таблице:

In [10]:
medicine_bn.cpt(medicine_bn.idFromName('Symptom1')).var_names

['Disease', 'Symptom1']

Соответственно, таблица выглядит так (переменная Disease - первая):

| Disease   | Symptom1 | p    |
|-----------|----------|------|
|  No       |    0     | 0.95  |
|  No       |    1     | 0.05  |
|  Disease1 |    0     | 0.1  |
|  Disease1 |    1     | 0.9  |
|  Disease2 |    0     | 0.6  |
|  Disease2 |    1     | 0.4  |

(в исходных данных есть только вероятности проявления симптома, вероятности его непроявления вычисляются как дополнение до 1)

In [11]:
medicine_bn.cpt(medicine_bn.idFromName('Symptom1')).fillWith([0.95, 0.05, 0.1, 0.9, 0.6, 0.4])

(gum::Potential<double>@0x203c9f5d5b0) 
      ||  Symptom1         |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9500  | 0.0500  |
Diseas|| 0.1000  | 0.9000  |
Diseas|| 0.6000  | 0.4000  |

Это же можно делать и с помощью словарей (пожалуй, наиболее читаемый способ):

In [12]:
symptom2 = medicine_bn.idFromName('Symptom2')

medicine_bn.cpt(symptom2)[{'Disease': 'No'}] = [0.95, 0.05]   # два значения: для Symptom2=0 и Symptom2=1 соответственно
medicine_bn.cpt(symptom2)[{'Disease': 'Disease1'}] = [0.7, 0.3]
medicine_bn.cpt(symptom2)[{'Disease': 'Disease2'}] = [0.2, 0.8]

medicine_bn.cpt(symptom2)

(gum::Potential<double>@0x203c9f5db40) 
      ||  Symptom2         |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9500  | 0.0500  |
Diseas|| 0.7000  | 0.3000  |
Diseas|| 0.2000  | 0.8000  |

Аналогично для результата диагностики. Сама переменная "результат диагностики" (Test) может принимать два значения: отрицательный (заболевание не выявлено, кодируется 0) и положительный (заболевание выявлено, кодируется 1). В соответствии с условием, у теста есть и ложные срабатывания, то есть вероятность того, что тест покажет положительный результат при отсутствии заболевания З2 не равна 0. Поскольку тест чувствителен только к заболеванию З2, вероятностные характеристики для отсутствия заболевания и для заболевания З1 одинаковы:

Существует также диагностическая процедура (тест), которая позволяет выявить заболевание З2 (как это обычно бывает, с некоторой точностью) - если у пациента действительно присутствует заболевание З2, то результат процедуры оказывается положительным в $95\%$ случаев, однако он также оказывается положительным в $3\%$ случаев, когда у пациента нет З2. Стоимость теста - 1200 руб.

In [13]:
test = medicine_bn.idFromName('Test')

medicine_bn.cpt(test)[{'Disease': 'No'}] = [0.97, 0.03]   # два значения: для Test=0 (отр.) и Test=1 (пол.)
medicine_bn.cpt(test)[{'Disease': 'Disease1'}] = [0.97, 0.03]
medicine_bn.cpt(test)[{'Disease': 'Disease2'}] = [0.05, 0.95]

medicine_bn.cpt(test)

(gum::Potential<double>@0x203c9f5dee0) 
      ||  Test             |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9700  | 0.0300  |
Diseas|| 0.9700  | 0.0300  |
Diseas|| 0.0500  | 0.9500  |

На этом спецификация сети завершена. Обратите внимание, что мы нигде не задали полезности исходов и "независимую" переменную, соответсвующую принимаемому решению (лечить или не лечить). Их просто нельзя представить с помощью формализма байесовских сетей (в отличие от диаграмм влияния). 

# Запросы

Проверим работу сети с помощью нескольких запросов.

Например, можно оценить вероятность заболевания при наличии тех или иных симптомов и/или результата теста:

In [14]:
# Создадим экземпляр "машины вывода".
# Здесь VariableElimination - это один из распространенных алгоритмов
# точного вывода на БС
ie=gum.VariableElimination(medicine_bn)

# Рассчет всех апостериорных вероятностей
ie.makeInference()

# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5dde0) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.9840  | 0.0100  | 0.0060  |

Очевидно, в данном случае апостериорная вероятность совпадает с априорной (мы не сообщили модели никаких новых данных по сравнению с теми, на которых основывается априорная вероятность).

In [15]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствует симптом 1 (про симптом 2 ничего не известно)
ie.setEvidence({'Symptom1': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5dc40) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.8119  | 0.1485  | 0.0396  |

Наличие информации о наличии симптома С1 не является особенно ценным. Видимо, все дело в том, что данный симптом относительно часто встречается и без заболеваний. Что насчет второго симптома?

In [16]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствует симптом 2 (про симптом 1 ничего не известно)
ie.setEvidence({'Symptom2': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5e0e0) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.8632  | 0.0526  | 0.0842  |

Наличие информации о наличии симптома С2 еще менее ценно. Хотя видно, что вероятность заболевания З2 при этом чуть выше, чем заболевания З1. Впрочем, обе достаточно малы, что может быть результатом того, что заболевания достаточно редкие (априорная вероятность мала). Что если наблюдаются оба симптома?

In [17]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствуют оба симптома (про тест ничего не известно)
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5e140) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.3475  | 0.3814  | 0.2712  |

При наличии обоих симптомов вероятности каждого из заболеваний и здорового состояния приблизительно равны. Здесь может помочь результат диагностики.

In [18]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствуют оба симптома и тест отрицательный
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1,
                'Test': 0})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5dcc0) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.4678  | 0.5134  | 0.0188  |

Отрицательный тест (при наличии симптомов) свидетельствует в пользу заболевания З1.

In [19]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствуют оба симптома и тест положительный
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1,
                'Test': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(gum::Potential<double>@0x203c9f5e000) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.0373  | 0.0409  | 0.9218  |

Положительный тест (в дополнение к наблюдаемым симптомам) позволяет с уверенностью судить о наличии заболевания З2.

# Определение ожидаемой полезности на основе байесовской сети

До данного момента мы рассуждали исключительно о вероятности той или иной ситуации (описываемой определенным значением переменных) при наличии свидетельств. Для каждой ситуации также может быть сопоставлена определенная полезность (привлекательность) с точки зрения пациента (например). Само сопоставление ситуации (значения набора переменных) и полезности (привлекательности) задано в условии. К сожалению, вероятностная природа задачи не позволяет говорить о том, что то или иное значение функции полезности будет получено при определенных наблюдениях. Однако возможность оценить вероятность каждой из ситуаций, фигурирующих в определении функции полезности, позволяет оценить _ожидаемую полезность_ (математическое ожидание полезности) при том или ином наборе свидетельств.


Одна из сложностей задачи связана с тем, что задано две метрики эффективности - деньги (стоимость терапии, стоимость диагностики) и "эффект" от применения терапии, который как-то связан с самочувствием. Простейшим способом принятия решений в условиях двух критериев является построение агрегированного критерия, являющегося сверткой базовых. Если идти по этому пути, то необходимо каким-то образом сопоставить единицу самочувствия пациента денежным затратам. Очевидно, это сопоставление не может быть универсальным и может сильно зависеть от индивидуальных предпочтений пацента. Поэтому при применении подобного подхода значение коэффициентов, как правило, остаются "подвижными" и иногда предлагаются специальные формальные процедуры для выяснения предпочтений пользователя (например, метод попарных сравнений). 

Поскольку в условии мало что сказано про "эффект", попробуем дополнить его на основе здравого смысла (различной степени "здравости"). Так, при отсутствии лечения "эффект" имеет значение около минус 100. Мы не знаем специфики заболеваний, но предположим, что это относительно "стандартные" вирусные или бактериальные инфекции, с которыми организм, как правило, может справиться самостоятельно. При отсутствии лечения подобные заболевания могут длиться около 2х недель. Соответственно, под значением "эффекта" $-100$ будем понимать нетрудоспособность в течение 14-ти дней (исходя из пятидневной рабочей недели). Исходя из среднего размера оплаты труда (70 т.р. в месяц) каждый день нетрудоспособности может быть оценен в 2333 руб., следовательно, единица "эффекта" может быть приравнена к 327 руб.

Подобная скаляризация позволяет записать функцию полезности следующим образом (колонка "Всего" показывает суммарную стоимость с учетом стоимости терапии - 2500 руб. и 3000 руб.):

| Болезнь | Терапия  |  Эффект  | Эффект (руб.) |    Всего (руб.) |
|---------|----------|----------|---------------|-----------------|
|  Нет    |  Нет     |     0    |      0        |         0       | 
|  Нет    |  Т1      |   -20    |    -6540      |     -9040       |
|  Нет    |  Т2      |   -25    |    -8175      |     -8475       |
|  З1     |  Нет     |   -100   |   -32700      |    -32700       |
|  З1     |  T1      |   -50    |   -16350      |    -18850       |
|  З1     |  T2      |   -125   |   -40875      |    -41175       |
|  З2     |  Нет     |   -110   |   -35970      |    -35970       |
|  З2     |  T1      |   -130   |   -42510      |    -45010       |
|  З2     |  T2      |   -45    |   -14715      |    -15015       |



Опишем эту таблицу с помощью структуры Potential (потенциал), предоставляемой библиотекой pyAgrum. Потенциал можно представить себе как многомерную таблицу. Эта абстракция широко используется при реализации различных алгоримов на вероятностных графических моделях.

In [20]:
# Потенциал определяется на наборе переменных. В нашем случае их две - 
# Болезнь (используем ее определение из сети) и Лечение.
d = medicine_bn.variable(0)
c = gum.LabelizedVariable('Treat', 'Treated?', 3)
c.changeLabel(0, 'No')
c.changeLabel(1, 'T1')
c.changeLabel(2, 'T2')

utility_potential = gum.Potential().add(c).add(d)
utility_potential.fillWith([     0,  -9040,  -8475, 
                            -32700, -18850, -41175, 
                            -35970, -45010, -15015])

(gum::Potential<double>@0x203c9f5dd40) 
      ||  Treat                      |
Diseas||No       |T1       |T2       |
------||---------|---------|---------|
No    || 0.0000  | -9040.0000| -8475.0000|
Diseas|| -32700.0000| -18850.0000| -41175.0000|
Diseas|| -35970.0000| -45010.0000| -15015.0000|

Каждой строчке таблицы выше (и, соответветственно, каждой ячейке потенциала) можно поставить в соответствие вероятность, которая определяется как совместная вероятность всех вероятностных переменных, участвующих в таблице (с учетом имеющихся свидетельств). В данном случае вероятностная переменная всего одна: Болезнь (Disease). Если ничего про пациента неизвестно, то вероятность значения переменной Болезнь - это просто априорная вероятность:

In [21]:
def disease_proba_factor(ie, ev):
    ie.eraseAllEvidence()
    ie.setEvidence(ev)
    return ie.posterior('Disease') #.toarray()

In [22]:
disease_proba_factor(ie, {})

(gum::Potential<double>@0x203c9f5de80) 
  Disease                    |
No       |Disease1 |Disease2 |
---------|---------|---------|
 0.9840  | 0.0100  | 0.0060  |

Ожидаемая полезность позволяет агрегировать значения полезности, связанные с вероятностными переменными. Она определяется следующим образом: 

$$
EU(Treat | E) = \sum_{d \in Disease}[p(d|E)*U(d, Treat)]
$$

Здесь $E$ - свидетельства, то, что известно в момент принятия решения.

То есть, необходимо суммировать произведения вероятности различных заболеваний при условии известных свидетельств ($p(d|E)$) на значение соответствующей функции полезности.

Подобная операция типична для вывода на вероятностных графических моделях и поддерживается непосредственно классом Potential:


In [23]:
# Произведение потенциалов эквивалентно произведению всех значений, у которых
# одинаковы значения соответствующих переменных (Disease),
# а margSumOut() осуществляет маржинализацию переменной Disease (сложение
# ячеек, соответствующих этой переменной для каждого сочетания других переменных)
(utility_potential * disease_proba_factor(ie, {})).margSumOut(['Disease'])

(gum::Potential<double>@0x203c9f5da20) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 -542.8200| -9353.9200| -8841.2400|

Таким образом, каждому из возможных значений управления ставится в соответствие определенное значение ожидаемой полезности.

В свою очередь, оптимальным решением является такое, которое максимизирует ожидаемую полезность:

$$
best = \arg\max_{t \in Treat} EU(t | E).
$$

Таким образом, оптимальным решением в таком случае является решение "Не лечить". А итоговой ожидаемой полезностью с учетом принятого решения будет -542.82.

Обобщим поиск итоговой ожидаемой полезности с учетом этих рассуждений.

In [24]:
# Этот хак обходит некорректную работу некоторых функций с отрицательными потенциалами
def max_u(potential):
    min_val = potential.min()
    if min_val < 0:
        potential = potential - (min_val - 1)
        return potential.max() + (min_val - 1)
    return potential.max()

def max_argmax_u(potential):
    min_val = potential.min()
    if min_val < 0:
        potential = potential - (min_val - 1)
        return potential.max() + (min_val - 1), potential.argmax()
    return potential.max(), potential.argmax()

def eu(ie, evidence, utility_potential):
    """Вычисление итоговой ожидаемой полезности с учетом свидетельств.
    
    Внимание! Корректна только для определенного вида функции полезности - 
              связывающей Болезнь и Лечение.
    """
    tmp = (utility_potential * disease_proba_factor(ie, evidence)).margSumOut(['Disease'])
    # Этот хак обходит некорректную работу некоторых функций с отрицательными потенциалами
    return max_argmax_u(tmp)

eu(ie, {}, utility_potential)

(-542.8199999999997, [{'Treat': 0}])

Что если мы знаем, что у пациента наблюдается только симптом С2? Стоит ли делать диагностическую процедуру и получить дополнительную информацию для принятия решения? В принципе, можно добавить решение о диагностической процедуре как еще одну переменную в потенциал полезности, однако при такой маленькой сети можно поступить и проще: оценить распределение вероятностей результата теста при имеющейся информации и возможные исходы (с учетом теста и без него):

1. Если тест не делаем:

In [25]:
(utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1})).margSumOut(['Disease'])

(gum::Potential<double>@0x203c9f5dfe0) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 -2271.7067| -11174.1466| -9048.8221|

То лечить не будем, ожидаемая полезность равна -2271,7.

2. Если тест делаем:

In [26]:
def test_proba_factor(ie, ev):
    """Вычисление вероятности показаний теста в зависимости от имеющейся информации."""
    ie.eraseAllEvidence()
    ie.setEvidence(ev)
    return ie.posterior('Test') #.toarray()

# Если тест делаем, то возможны два исхода (не забываем учесть стоимость теста - 1200 руб.): 
# 1. Тест окажется положительным
test_positive = (utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1, 'Test': 1})).margSumOut(['Disease']) - 1200 
test_positive

(gum::Potential<double>@0x203c9f5da60) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 -25001.1719| -33991.4974| -14060.5469|

(оптимальным решением будет применение терапии T2, что приведет к ожидаемой полезности -14060.54)

In [27]:
# 2. Тест может оказаться и отрицательным
test_negative = (utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1, 'Test': 0})).margSumOut(['Disease']) - 1200
test_negative

(gum::Potential<double>@0x203c9f5dd80) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 -1521.0505| -10415.5276| -9903.4645|

(в этом случае оптимальным решением будет отсутствие лечения, что приведет к ожидаемой полезности -1521.05)

Вероятности этих исходов различны:

In [28]:
tmp = test_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1})
tmp

(gum::Potential<double>@0x203c9f5df80) 
  Test             |
0        |1        |
---------|---------|
 0.9169  | 0.0831  |

In [29]:
# Таким образом, итоговая ожидаемая полезность:
tmp[0]*max_u(test_negative) + tmp[1]*max_u(test_positive)

-2562.79326923077

Ожидаемая полезность с учетом знания результатов теста оказалась ниже, чем без учета знания результатов теста. Следовательно, тест в такой ситуации проводить нецелесообразно. Любопытно, что разница в ожидаемой полезности оказалась всего около 200 руб., то есть был бы тест чуть дешевле, его проводить в такой ситуации оказалось бы выгодно.

# Цена информации

По определению, цена информации $e'$ - это разница между полезностью с учетом информации и полезностью без ее учета:

$$
IV = EU(e \cup e') - EU(e)
$$

Смысл здесь в том, что наличие определенной информации (не определенные значения, а сама возможность доступа к значению тех или иных переменных) позволяет сделать стратегию принятия решения более выразительной - можно принимать различные решения в зависимости от значений наблюдаемых переменных.

## Оценка цены информации о симптоме С1

Чтобы провести оценку цены информации необходимо построить оптимальную стратегию принятия решения с учетом симптома С1, а потом оценить ожидаемую полезность от следования такой стратегии.

Построим потенциал, совмещающий информацию, имеющуюся при принятии решения, и соответствующие значения функции полезности:


In [30]:
d = medicine_bn.variable(0)   # Болезнь
s1 = medicine_bn.variable(1)  # Симптом 1
c = gum.LabelizedVariable('Treat', 'Treated?', 3) # Лечение
c.changeLabel(0, 'No')
c.changeLabel(1, 'T1')
c.changeLabel(2, 'T2')

utility_potential_s1 = gum.Potential().add(c).add(s1).add(d)

# Значение полезности по факту не зависит от наличия симптома 
# просто копируем значения из таблицы utility_potential
for disease in ['No', 'Disease1', 'Disease2']:
    for s1 in [0, 1]:
        for t in ['No', 'T1', 'T2']:
            utility_potential_s1[{'Disease': disease, 'Symptom1': s1, 'Treat': t}] = utility_potential[{'Disease': disease, 'Treat': t}]
utility_potential_s1    

(gum::Potential<double>@0x203c9f5dea0) 
             ||  Treat                      |
Sympto|Diseas||No       |T1       |T2       |
------|------||---------|---------|---------|
0     |No    || 0.0000  | -9040.0000| -8475.0000|
1     |No    || 0.0000  | -9040.0000| -8475.0000|
0     |Diseas|| -32700.0000| -18850.0000| -41175.0000|
1     |Diseas|| -32700.0000| -18850.0000| -41175.0000|
0     |Diseas|| -35970.0000| -45010.0000| -15015.0000|
1     |Diseas|| -35970.0000| -45010.0000| -15015.0000|

Далее необходимо найти вероятность, соответствующую каждой ячейке. Это совместная вероятность $p(Disease, Symptom1) = p(Symptom1|Disease)p(Disease)$. Соответствующие значения содержатся непосредственно в таблицах условных распределений:

In [31]:
joint_proba_d_s1 = medicine_bn.cpt(medicine_bn.idFromName('Disease')) * medicine_bn.cpt(medicine_bn.idFromName('Symptom1'))

Маржинализация переменной Болезнь даст ожидаемую полезность для каждого сочетания наблюдаемой и управляемой переменных:

In [32]:
(utility_potential_s1 * joint_proba_d_s1).margSumOut(['Disease'])

(gum::Potential<double>@0x203c9f5e120) 
      ||  Symptom1         |
Treat ||0        |1        |
------||---------|---------|
No    || -162.1920| -380.6280|
T1    || -8631.4780| -722.4420|
T2    || -8017.6590| -823.5810|

Видно, что вне зависимости от наличия симптома, оптимальным управлением будет отсутствие лечения. Следовательно, при наличии информации о симптоме С1 стратегия будет полностью совпадать со стратегией без этой информации, а значит и ожидаемая полезность не изменится. Таким образом, цена информации о симптоме С1 равна 0.

# Рекомендуемые ссылки:

- https://webia.lip6.fr/~phw//aGrUM/docs/last/notebooks/01-tutorial.ipynb.html и другие обучающие материалы по PyAgrum.
